In [145]:
import os
import numpy as np
import cv2
from scipy.ndimage import gaussian_filter

In [146]:
# Set the random seed for reproducibility
gt_path = "YOUR_PATH/gt"
input_path = "YOUR_PATH/input"
txt_path = "YOUR_PATH/txt"
os.makedirs(gt_path, exist_ok=True)
os.makedirs(input_path, exist_ok=True)
os.makedirs(txt_path, exist_ok=True)

In [147]:
#  Define the parameters
output_size = (256,256)

In [148]:
# Generate the images
def generate_lines_image():
    height, width = 256,256
    img = np.zeros((height, width), dtype=np.uint8)
    gt_data = []
    used_columns = set()
    
    # generate random vertical lines
    num_lines = np.random.randint(10, 31)
    for _ in range(num_lines):
        max_attempts = 100  # maximum attempts to place a line
        attempts = 0
        while attempts < max_attempts:
            attempts += 1
            x1 = np.random.randint(0, width)  
            # check if the column is already used
            if all(abs(x1 - used_col) >= 3 for used_col in used_columns):
                y1 = np.random.randint(0, 50)  
                length = np.random.randint(200, 401)  
                x2, y2 = x1, min(y1 + length, height - 1)  
                gt_data.append(((x1, y1), (x2, y2)))
                cv2.line(img, (x1, y1), (x2, y2), 255, 1)
                used_columns.add(x1) 
                break
        else:
            print(f"Warning: Could not place a line after {max_attempts} attempts.")
            
        
    # generate random horizontal lines
    for _ in range(np.random.randint(5, 11)):
        x1, y1 = np.random.randint(0, width), np.random.randint(0, height)
        length = np.random.randint(20, 31)
        angle = np.random.uniform(0, 2 * np.pi)
        x2 = int(x1 + length * np.cos(angle))
        y2 = int(y1 + length * np.sin(angle))
        x2 = np.clip(x2, 0, width - 1)
        y2 = np.clip(y2, 0, height - 1)
        gt_data.append(((x1, y1), (x2, y2)))
        cv2.line(img, (x1, y1), (x2, y2), 255, 1)

    return img, gt_data

# resize the image
def generate_lines_image_resized():
    img, gt_data = generate_lines_image()
    resized_img = cv2.resize(img, output_size, interpolation=cv2.INTER_NEAREST)
    return resized_img, gt_data

In [149]:
# save the image and ground truth data
def save_line_data_to_txt(filename, gt_data):
    with open(filename, "w") as f:
        for line in gt_data:
            (x1, y1), (x2, y2) = line
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi  # 计算角度
            length = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)    # 计算像素点数量 (即长度)
            f.write(f"Angle: {angle:.2f}, Length: {int(length)}\n")

In [150]:
# apply brightness and blur
def apply_brightness_and_blur(img, gt_data):
    blurred_img = np.zeros_like(img, dtype=np.float32)
    for line in gt_data:
        ((x1, y1), (x2, y2)) = line
        brightness = np.random.randint(205, 256)
        temp_img = np.zeros_like(img, dtype=np.uint8)
        cv2.line(temp_img, (x1, y1), (x2, y2), brightness, 1)
        sigma = np.random.uniform(0.8, 2.4)
        temp_img = gaussian_filter(temp_img.astype(np.float32), sigma=sigma)
        blurred_img += temp_img
    return np.clip(blurred_img, 0, 255).astype(np.uint8)

In [151]:
# add background and noise
def add_background_and_noise(img):
    height, width = img.shape
    background_gray = np.random.randint(30, 121)
    background = np.full((height, width), background_gray, dtype=np.uint8)
    combined = cv2.add(background, img)

    # add noise
    signal = img.max()
    noise_std = signal / np.random.uniform(20,40)
    noise = np.random.normal(0, noise_std, size=(height, width))
    noisy_img = combined + noise
    return np.clip(noisy_img, 0, 255).astype(np.uint8)

In [152]:
# add light stripes background
def add_light_stripes_background(img):
    height, width = img.shape
    # add random light stripes
    striped_background = np.zeros_like(img, dtype=np.float32)
    num_stripes = np.random.randint(5, 10)
    for i in range(num_stripes):
        intensity = np.random.randint(0, 5)  
        stripe_height = np.random.randint(height // 10, height // 5)  
        stripe_y_start = np.random.randint(0, height - stripe_height)  
        stripe = np.linspace(intensity, intensity + 5, stripe_height)
        for y in range(stripe_height):
            striped_background[stripe_y_start + y, :] = stripe[y]

    # apply Gaussian blur to the stripes
    final_img = img.astype(np.float32) + striped_background
    return np.clip(final_img, 0, 255).astype(np.uint8)

In [153]:
# main
for i in range(1): 
    base_img, gt_data = generate_lines_image_resized()
    gt_save_path = os.path.join(gt_path, f"arraygt_{i:04d}.png")
    cv2.imwrite(gt_save_path, base_img)

    blurred_img = apply_brightness_and_blur(base_img, gt_data)
    noisy_img = add_background_and_noise(blurred_img)
    
    # add light stripes background
    final_img = add_light_stripes_background(noisy_img)

    input_save_path = os.path.join(input_path, f"arrayinput_{i:04d}.png")
    cv2.imwrite(input_save_path, final_img)

    # save the ground truth data to a text file
    txt_filename = os.path.join(txt_path, f"arrayinput_{i:04d}.txt")
    save_line_data_to_txt(txt_filename, gt_data)